table of contents
1. Prepare
2. Data Import, Encoding
3. EDA
4. write df

In [1]:
import os, random, time

import numpy as np
import pandas as pd

# Prepare

## Define parameters

In [2]:
base_dir = os.path.join('..','..')
data_dir = os.path.join(base_dir, 'data')
raw_dir = os.path.join(data_dir, 'raw')
processed_dir = os.path.join(data_dir, 'processed')

infile_list = os.listdir(processed_dir)
for i, infile in enumerate(infile_list):
    print(i, infile)

0 events.csv
1 matches
2 passing.zip.bkp
3 events
4 passing.csv
5 coaches.csv
6 passing_time.csv
7 .~lock.teams.csv#
8 passing.zip
9 passing_match.csv
10 teams.csv


## Data import

In [3]:
competition = 'Spain'
#df = pd.read_csv(os.path.join(processed_dir+'/events', 'events_'+competition+'.csv'))
df = pd.read_json(os.path.join(raw_dir+'/events', 'events_'+competition+'.json'))
teams_df = pd.read_json(os.path.join(raw_dir, 'teams.json'))
players_df = pd.read_json(os.path.join(raw_dir, 'players.json'))
matches_df = pd.read_json(os.path.join(raw_dir+'/matches', 'matches_'+competition+'.json'))

In [4]:
#players_df['shortName'] = players_df.shortName.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))
#teams_df['officialName'] = teams_df.officialName.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))
#teams_df['name'] = teams_df.name.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))


In [5]:
df.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422


In [6]:
df[['st_x', 'st_y', 'ed_x', 'ed_y']] = df.apply(lambda xs: [xs['positions'][0]['x'], xs['positions'][0]['y'], xs['positions'][1]['x'], xs['positions'][1]['y']] if (len(xs['positions']) >= 2) else [xs['positions'][0]['x'], xs['positions'][0]['y'],None, None] , result_type='expand', axis=1)

In [7]:
teams_df_indexed = teams_df.set_index('wyId')

df = pd.merge(df, teams_df_indexed, left_on='teamId', right_index=True)

In [8]:
players_df_indexed = players_df.set_index('wyId')

df = pd.merge(df, players_df_indexed, left_on='playerId', right_index=True)

In [9]:
matches_df_indexed = matches_df.set_index('wyId')

df = pd.merge(df, matches_df_indexed, left_on='matchId', right_index=True)

In [10]:
#df.loc[df.eventName=='Shot'].head(30)
df.iloc[1890]

eventId                                                                  8
subEventName                                                   Simple pass
tags                                                        [{'id': 1801}]
playerId                                                             25472
positions                           [{'y': 3, 'x': 82}, {'y': 6, 'x': 67}]
matchId                                                            2565558
eventName                                                             Pass
teamId                                                                 682
matchPeriod                                                             1H
eventSec                                                            1618.6
subEventId                                                              85
id                                                               182021285
st_x                                                                    82
st_y                     

In [11]:
df.st_x.dtype

dtype('float64')

In [12]:
df_pass = df.loc[df.eventName=='Pass'].copy()

In [25]:
df_pass[df_pass.name=='Barcelona'].head()
df_pass.iloc[243]

eventId                                                                  8
subEventName                                                   Simple pass
tags                                                        [{'id': 1801}]
playerId                                                              3277
positions                         [{'y': 39, 'x': 38}, {'y': 70, 'x': 35}]
matchId                                                            2565548
eventName                                                             Pass
teamId                                                                 682
matchPeriod                                                             1H
eventSec                                                           549.464
subEventId                                                              85
id                                                               180864501
st_x                                                                    38
st_y                     

In [55]:
#df_pass['st_x'] = None
#df_pass['st_y'] = None
#df_pass['ed_x'] = None
#df_pass['ed_y'] = None

#df_pass['list_positions'] = df_pass['positions'].apply(lambda x: list(x))

#df_pass['list_positions'].values
#df_pass[['st_x', 'st_y', 'ed_x', 'ed_y']] = df_pass.apply(lambda xs: [xs['positions'][0]['x'], xs['positions'][0]['y'], xs['positions'][1]['x'], xs['positions'][1]['y']], result_type='expand', axis=1)
#df_pass['st_x'] = df_pass.apply(lambda xs: xs['positions'][0]['x'])


df_pass['success'] = df_pass.apply(lambda xs: 1 if 1801 in [tag['id'] for tag in xs['tags']] else 0, axis=1)

In [56]:
df_pass[df_pass.shortName=='Jordi Alba'].head(20)[['name', 'lastName','st_x', 'st_y']]

,name,lastName,st_x,st_y
235215,Barcelona,Alba Ramos,32.0,7.0
235217,Barcelona,Alba Ramos,33.0,8.0
235221,Barcelona,Alba Ramos,35.0,6.0
235240,Barcelona,Alba Ramos,52.0,7.0
235255,Barcelona,Alba Ramos,57.0,5.0
235265,Barcelona,Alba Ramos,39.0,6.0
235275,Barcelona,Alba Ramos,60.0,5.0
235277,Barcelona,Alba Ramos,66.0,10.0
235300,Barcelona,Alba Ramos,54.0,21.0
235316,Barcelona,Alba Ramos,68.0,5.0


## Match Result

In [57]:
df_pass[['won','drawn', 'lost']] = df_pass.apply(lambda x: [1 if x['winner']==x['teamId'] else 0, 1 if x['winner']==0 else 0, 1 if (x['winner']!=x['teamId'] and x['winner']!=0) else 0], result_type='expand', axis=1)

### Calculate sectors of the playing field

Divide the land into 18 rectangles, we have a grid of 6 x 3 that mark 18 zones (https://footballbh.net/2019/02/04/the-use-of-zone-14-the-key-to-winning-football-matches/)

Zones

* Horizontal zone 
    * left 
    * center 
    * right
* Vertical zone  
    * own field (low, middle, high) 
    * rival field (low, middle, high)


In [67]:
#sector           = ('own', 'rival')
vertical_zones   = (('left',(0,25)),('middle', (25,75)),('right', (75,100)))
horizontal_zones = (('own_low', (0, 17)), ('own_middle', (17, 30)), ('own_high', (30, 50)),('rival_low', (50, 70)), ('rival_middle', (70, 83)), ('rival_high', (83, 100)))


#df_pass[list(map(lambda v : 'v_'+ v[0], vertical_zones))] = df_pass.apply(lambda x: list(map(lambda v : v[1], vertical_zones)), result_type='expand', axis=1)


In [68]:
df_pass[list(map(lambda v : 'v_'+ v[0], vertical_zones))] = df_pass.apply(lambda x: list(map(lambda v : 1 if (v[1][0] < x['st_y'] <= v[1][1]) else 0, vertical_zones)), result_type='expand', axis=1)

In [69]:
df_pass[list(map(lambda v : 'h_'+ v[0], horizontal_zones))] = df_pass.apply(lambda x: list(map(lambda v : 1 if (v[1][0] < x['st_x'] <= v[1][1]) else 0, horizontal_zones)), result_type='expand', axis=1)

In [70]:
df_pass[df_pass.shortName=='Jordi Alba'].head(20)[['name', 'lastName','st_x', 'st_y', 'v_left']]

,name,lastName,st_x,st_y,v_left
235215,Barcelona,Alba Ramos,32.0,7.0,1
235217,Barcelona,Alba Ramos,33.0,8.0,1
235221,Barcelona,Alba Ramos,35.0,6.0,1
235240,Barcelona,Alba Ramos,52.0,7.0,1
235255,Barcelona,Alba Ramos,57.0,5.0,1
235265,Barcelona,Alba Ramos,39.0,6.0,1
235275,Barcelona,Alba Ramos,60.0,5.0,1
235277,Barcelona,Alba Ramos,66.0,10.0,1
235300,Barcelona,Alba Ramos,54.0,21.0,1
235316,Barcelona,Alba Ramos,68.0,5.0,1


In [71]:
df_pass[df_pass.shortName=='Jordi Alba'].head(20)[['name', 'lastName','st_x', 'st_y', 'v_left']]

,name,lastName,st_x,st_y,v_left
235215,Barcelona,Alba Ramos,32.0,7.0,1
235217,Barcelona,Alba Ramos,33.0,8.0,1
235221,Barcelona,Alba Ramos,35.0,6.0,1
235240,Barcelona,Alba Ramos,52.0,7.0,1
235255,Barcelona,Alba Ramos,57.0,5.0,1
235265,Barcelona,Alba Ramos,39.0,6.0,1
235275,Barcelona,Alba Ramos,60.0,5.0,1
235277,Barcelona,Alba Ramos,66.0,10.0,1
235300,Barcelona,Alba Ramos,54.0,21.0,1
235316,Barcelona,Alba Ramos,68.0,5.0,1


In [72]:
#df_pass.reindex()
df_pass.groupby(['matchId', 'teamId', 'eventId','subEventId', 'matchPeriod']).v_left.sum().head(30)


matchId  teamId  eventId  subEventId  matchPeriod
2565548  682     8        80          1H              0
                                      2H              2
                          81          2H              0
                          82          1H              4
                                      2H              4
                          83          1H              4
                                      2H              7
                          84          1H              0
                          85          1H             29
                                      2H             59
                          86          1H              1
                                      2H              3
         695     8        80          1H              8
                                      2H              4
                          81          2H              0
                          82          1H              2
                                      2H              

## Export 

In [73]:
fields = ['gameweek','matchId', 'teamId', 'name', 'playerId', 'shortName', 'eventName', 'subEventName', 'matchPeriod', 'eventSec' , 'st_x', 'st_y', 'ed_x', 'ed_y', 'success'] 
fields +=list(map(lambda v : 'v_'+ v[0], vertical_zones)) 
fields +=list(map(lambda v : 'h_'+ v[0], horizontal_zones))
fields +=['won','drawn', 'lost']

fields

['gameweek',
 'matchId',
 'teamId',
 'name',
 'playerId',
 'shortName',
 'eventName',
 'subEventName',
 'matchPeriod',
 'eventSec',
 'st_x',
 'st_y',
 'ed_x',
 'ed_y',
 'success',
 'v_left',
 'v_middle',
 'v_right',
 'h_own_low',
 'h_own_middle',
 'h_own_high',
 'h_rival_low',
 'h_rival_middle',
 'h_rival_high',
 'won',
 'drawn',
 'lost']

In [74]:
df_save = df_pass[fields].sort_values(by=['matchId','eventSec'])

In [75]:
df_save_match = df_pass[fields].loc[df_pass.matchId==2565548].sort_values(by=['eventSec'])

In [76]:
df_pass.head(50)

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,...,v_right,h_own_low,h_own_middle,h_own_high,h_rival_low,h_rival_middle,h_rival_high,success,qtime,qtime2
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,...,0,0,0,1,0,0,0,1,"(-1, 4]","(-1.0, 4.0]"
39,8,Simple pass,[{'id': 1801}],3542,"[{'y': 48, 'x': 39}, {'y': 65, 'x': 26}]",2565548,Pass,682,1H,166.859710,...,0,0,0,1,0,0,0,1,"(-1, 4]","(-1.0, 4.0]"
43,8,High pass,"[{'id': 901}, {'id': 1802}]",3542,"[{'y': 67, 'x': 32}, {'y': 42, 'x': 74}]",2565548,Pass,682,1H,181.665639,...,0,0,0,1,0,0,0,0,"(-1, 4]","(-1.0, 4.0]"
50,8,Simple pass,[{'id': 1801}],3542,"[{'y': 60, 'x': 35}, {'y': 97, 'x': 42}]",2565548,Pass,682,1H,213.736626,...,0,0,0,1,0,0,0,1,"(-1, 4]","(-1.0, 4.0]"
60,8,Head pass,[{'id': 1801}],3542,"[{'y': 25, 'x': 26}, {'y': 9, 'x': 27}]",2565548,Pass,682,1H,278.645906,...,0,0,1,0,0,0,0,1,"(4, 9]","(4.0, 9.0]"
85,8,Smart pass,"[{'id': 901}, {'id': 1802}]",3542,"[{'y': 61, 'x': 38}, {'y': 47, 'x': 54}]",2565548,Pass,682,1H,354.921472,...,0,0,0,1,0,0,0,0,"(4, 9]","(4.0, 9.0]"
105,8,Simple pass,[{'id': 1801}],3542,"[{'y': 71, 'x': 32}, {'y': 96, 'x': 42}]",2565548,Pass,682,1H,440.315069,...,0,0,0,1,0,0,0,1,"(4, 9]","(4.0, 9.0]"
199,8,Simple pass,[{'id': 1801}],3542,"[{'y': 77, 'x': 45}, {'y': 72, 'x': 32}]",2565548,Pass,682,1H,844.316154,...,1,0,0,1,0,0,0,1,"(14, 19]","(14.0, 19.0]"
205,8,Simple pass,[{'id': 1801}],3542,"[{'y': 87, 'x': 42}, {'y': 38, 'x': 38}]",2565548,Pass,682,1H,862.919269,...,1,0,0,1,0,0,0,1,"(14, 19]","(14.0, 19.0]"
240,8,Simple pass,[{'id': 1801}],3542,"[{'y': 23, 'x': 30}, {'y': 19, 'x': 30}]",2565548,Pass,682,1H,1004.295944,...,0,0,1,0,0,0,0,1,"(14, 19]","(14.0, 19.0]"


In [77]:
df_pass[df_pass.shortName=='Jordi Alba'].head(20)[['name', 'lastName','st_x', 'st_y', 'v_left', 'h_own_low', 'h_own_middle', 'h_own_high',]]

,name,lastName,st_x,st_y,v_left,h_own_low,h_own_middle,h_own_high
235215,Barcelona,Alba Ramos,32.0,7.0,1,0,0,1
235217,Barcelona,Alba Ramos,33.0,8.0,1,0,0,1
235221,Barcelona,Alba Ramos,35.0,6.0,1,0,0,1
235240,Barcelona,Alba Ramos,52.0,7.0,1,0,0,0
235255,Barcelona,Alba Ramos,57.0,5.0,1,0,0,0
235265,Barcelona,Alba Ramos,39.0,6.0,1,0,0,1
235275,Barcelona,Alba Ramos,60.0,5.0,1,0,0,0
235277,Barcelona,Alba Ramos,66.0,10.0,1,0,0,0
235300,Barcelona,Alba Ramos,54.0,21.0,1,0,0,0
235316,Barcelona,Alba Ramos,68.0,5.0,1,0,0,0


In [78]:
minute=5
end=50

#df_save_time = df_pass.groupby(['matchId', 'teamId', 'eventId','subEventId', 'matchPeriod', pd.cut(df_pass.eventSec/60, np.arange(-1, end, minute))]).agg({'v_left': 'sum', 'v_middle': 'sum'}).reset_index()

from sklearn.preprocessing import LabelEncoder
sc = LabelEncoder()
df_pass['qtime'] = pd.cut(df_pass.eventSec/60, np.arange(-1, end, minute))
df_pass['qtime2'] = df_pass['qtime'].astype(str)
df_save_time = df_pass.groupby(['matchId', 'teamId', 'eventId','subEventId', 'matchPeriod', 'qtime2']).agg({'eventSec':'min', 'v_left': 'sum', 'v_middle': 'sum', 'v_right': 'sum'})

#df_save_time = df_pass.groupby(['matchId', 'teamId', 'eventId','subEventId', 'matchPeriod', pd.cut(df_pass.eventSec/60, np.arange(-1, end, minute))])

df_save_time.reset_index(inplace=True)
df_save_time.sort_values(['matchId', 'matchPeriod', 'eventSec'], inplace=True)
df_save_time.head(5)

#.agg({'v_left': 'sum', 'v_middle': 'sum'})

,matchId,teamId,eventId,subEventId,matchPeriod,qtime2,eventSec,v_left,v_middle,v_right
44,2565548,682,8,85,1H,"(-1.0, 4.0]",2.994582,4,15,4
65,2565548,682,8,86,1H,"(-1.0, 4.0]",39.291345,0,0,1
0,2565548,682,8,80,1H,"(-1.0, 4.0]",40.568891,0,0,1
127,2565548,695,8,85,1H,"(-1.0, 4.0]",42.500082,3,3,1
148,2565548,695,8,86,1H,"(-1.0, 4.0]",150.448925,0,0,1


In [79]:
df_pass.sort_values(['matchId', 'matchPeriod', 'eventSec'])[(df_pass.matchId == 2565548) & (df_pass.qtime2 == '(-1.0, 4.0]')][['qtime', 'v_left']].head(20)

/home/diego/anaconda3/envs/wyscout/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,qtime,v_left
0,"(-1, 4]",0
1,"(-1, 4]",0
2,"(-1, 4]",0
3,"(-1, 4]",1
4,"(-1, 4]",0
5,"(-1, 4]",0
6,"(-1, 4]",0
7,"(-1, 4]",0
8,"(-1, 4]",0
9,"(-1, 4]",0


In [80]:
df_pass[['qtime', 'qtime2']]

,qtime,qtime2
0,"(-1, 4]","(-1.0, 4.0]"
39,"(-1, 4]","(-1.0, 4.0]"
43,"(-1, 4]","(-1.0, 4.0]"
50,"(-1, 4]","(-1.0, 4.0]"
60,"(4, 9]","(4.0, 9.0]"
...,...,...
15690,"(34, 39]","(34.0, 39.0]"
15694,"(34, 39]","(34.0, 39.0]"
15781,"(39, 44]","(39.0, 44.0]"
15845,"(44, 49]","(44.0, 49.0]"


In [81]:
df_save_time.head(100)

,matchId,teamId,eventId,subEventId,matchPeriod,qtime2,eventSec,v_left,v_middle,v_right
44,2565548,682,8,85,1H,"(-1.0, 4.0]",2.994582,4,15,4
65,2565548,682,8,86,1H,"(-1.0, 4.0]",39.291345,0,0,1
0,2565548,682,8,80,1H,"(-1.0, 4.0]",40.568891,0,0,1
127,2565548,695,8,85,1H,"(-1.0, 4.0]",42.500082,3,3,1
148,2565548,695,8,86,1H,"(-1.0, 4.0]",150.448925,0,0,1
...,...,...,...,...,...,...,...,...,...,...
138,2565548,695,8,85,2H,"(14.0, 19.0]",851.445525,7,7,1
55,2565548,682,8,85,2H,"(14.0, 19.0]",899.477960,4,5,2
83,2565548,695,8,80,2H,"(14.0, 19.0]",901.280759,0,1,0
8,2565548,682,8,81,2H,"(14.0, 19.0]",954.239649,0,1,0


## Save file

In [82]:
df_save.to_csv(os.path.join(processed_dir, 'passing.csv'), index=False)

In [83]:
df_save_match.to_csv(os.path.join(processed_dir, 'passing_match.csv'), index=False)

In [84]:
df_save_time.to_csv(os.path.join(processed_dir, 'passing_time.csv'), index=False)

NameError: name '__version__' is not defined